# DETECTR 049
This notebook is for formatting, analyzing and plotting the DETECTR_049 data

## Objective
To test NANCY DREW assay with AMP GLO, ENLITEN, and luminometer!

## Formatting to CSV file
Take txt file from plate reader and turn into tidy data format

In [28]:
#import needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

import holoviews as hv
import bokeh
import hvplot.pandas
import re

In [2]:
def read_Promega_injector(filename, experiment_name):
    #read in csv and extract read information
    DETECTR_raw = pd.read_csv(filename, header = None,skiprows=9,index_col=None)
    DETECTR_raw_read = pd.read_csv(filename, header = None,skipfooter=14,skiprows=4,index_col=None, engine = 'python').loc[0,2]
    
    #initialize clean dataframe
    DETECTR_cleaned = pd.DataFrame(columns = ['experiment','read','read_number','row','column', 'well','rfu_value'])

    #iterate over file to pull rfu_value and attach row, column and metadata
    for j in list(range(2, 14)):
        for k in list(range(1, 9)):
            experiment = experiment_name
            read = DETECTR_raw_read
            read_number=1
            rfu_value = DETECTR_raw.loc[k,j]
            row = DETECTR_raw.loc[k,1]
            column = DETECTR_raw.loc[0,j]
            well = row+str(int(column))
            DETECTR_cleaned.loc[len(DETECTR_cleaned)] = [experiment, read, read_number,row, column,well,rfu_value]
            
    #remove X values
    DETECTR_cleaned = DETECTR_cleaned.loc[DETECTR_cleaned['rfu_value'] != 'X'].reset_index(drop = True)
            
    return DETECTR_cleaned

In [3]:
def read_Promega_2injector(filename, injection1_name, injection2_name):
    #INJECTOR 1
    #read in csv and extract read information
    DETECTR_raw = pd.read_csv(filename, header = None,skiprows=9,skipfooter = 5,index_col=None, engine = 'python')
    DETECTR_raw_read = pd.read_csv("../raw_txt_files/DETECTR_044/ump0hr_amy-enliten-detectr044_12-6-2019_7-35-22 PM.csv", header = None,skipfooter=29,skiprows=4,index_col=None, engine = 'python')
    
    #initialize clean dataframe
    DETECTR_cleaned = pd.DataFrame(columns = ['experiment','read','read_number','row','column', 'well','rfu_value'])

    #iterate over file to pull rfu_value and attach row, column and metadata
    for j in list(range(2, 14)):
        for k in list(range(1, 9)):
            experiment = injection1_name
            read = DETECTR_raw_read.loc[0,2]
            read_number = 1
            rfu_value = DETECTR_raw.loc[k,j]
            row = DETECTR_raw.loc[k,1]
            column = DETECTR_raw.loc[0,j]
            well = row+str(int(column))
            DETECTR_cleaned.loc[len(DETECTR_cleaned)] = [experiment, read, read_number, row, column,well,rfu_value]
            
    for j in list(range(2, 14)):
        for k in list(range(11, 19)):
            experiment = injection2_name
            read = DETECTR_raw_read.loc[0,3]
            read_number = 2
            rfu_value = DETECTR_raw.loc[k,j]
            row = DETECTR_raw.loc[k,1]
            column = DETECTR_raw.loc[0,j]
            well = row+str(int(column))
            DETECTR_cleaned.loc[len(DETECTR_cleaned)] = [experiment, read, read_number,row, column,well,rfu_value]
            
    #remove X values
    DETECTR_cleaned = DETECTR_cleaned.loc[DETECTR_cleaned['rfu_value'] != 'X'].reset_index(drop = True)
            
    return DETECTR_cleaned

In [4]:
def read_Promega_endpoint(filename, experiment_name):
    #read in csv and extract read information
    DETECTR_raw = pd.read_csv(filename, header = None,skiprows=9,index_col=None)
    DETECTR_raw_read = pd.read_csv(filename, header = None,skipfooter=14,skiprows=4,index_col=None, engine = 'python').loc[0,1]
    
    #initialize clean dataframe
    DETECTR_cleaned = pd.DataFrame(columns = ['experiment','read','read_number','row','column', 'well','rfu_value'])

    #iterate over file to pull rfu_value and attach row, column and metadata
    for j in list(range(2, 14)):
        for k in list(range(1, 9)):
            experiment = experiment_name
            read = DETECTR_raw_read
            read_number = 1
            rfu_value = DETECTR_raw.loc[k,j]
            row = DETECTR_raw.loc[k,1]
            column = DETECTR_raw.loc[0,j]
            well = row+str(int(column))
            DETECTR_cleaned.loc[len(DETECTR_cleaned)] = [experiment, read, read_number, row, column,well,rfu_value]
            
    #remove X values
    DETECTR_cleaned = DETECTR_cleaned.loc[DETECTR_cleaned['rfu_value'] != 'X'].reset_index(drop = True)
            
    return DETECTR_cleaned

In [41]:
def read_Promega_kinetics(filename, experiment_name):
    #read in csv and extract read information
    DETECTR_raw = pd.read_csv(filename, header = None,skipfooter=1,skiprows=8,index_col=None, engine = 'python')
    DETECTR_raw_read = pd.read_csv(filename, header = None,skipfooter=101,skiprows=6,index_col=None, engine = 'python').loc[0,2]
    #initialize clean dataframe
    DETECTR_cleaned = pd.DataFrame(columns = ['experiment','read','read_number','row','column', 'well','rfu_value'])

    #iterate over file to pull rfu_value and attach row, column and metadata
    for j in list(range(1, len(DETECTR_raw))):
        for k in list(range(2, len(DETECTR_raw.columns))):
            experiment = experiment_name
            read = DETECTR_raw_read
            read_number = k-1
            rfu_value = DETECTR_raw.loc[j,k]
            row = str(DETECTR_raw.loc[j,1])[0]
            column = re.search('(?<=\D)\d+',DETECTR_raw.loc[j,1]).group(0) #DETECTR_raw.loc[j,1])[1:2]
            well = DETECTR_raw.loc[j,1]
            DETECTR_cleaned.loc[len(DETECTR_cleaned)] = [experiment, read, read_number, row, column,well,rfu_value]
            
    #remove X values
    DETECTR_cleaned = DETECTR_cleaned.loc[pd.notna(DETECTR_cleaned['rfu_value'])].reset_index(drop = True)
    #DETECTR_cleaned = DETECTR_cleaned.notna('rfu_value')
  
    return DETECTR_cleaned

In [60]:
ENLITEN_kinetics = read_Promega_kinetics("../raw_txt_files/DETECTR_049/nanci-amp_amyl-detectr049_1-9-2020_9-37-42 PM.csv", "NANCY_DREW")
ENLITEN_kinetics = ENLITEN_kinetics.assign(standard_substrate = len(ENLITEN_kinetics)*['AMP'])
ENLITEN_kinetics = ENLITEN_kinetics.assign(kit = len(ENLITEN_kinetics)*['AMP_no_kinase_ENLITEN'])
ENLITEN_kinetics = ENLITEN_kinetics.assign(sample_set = 8*(18*['NANCY_DREW']+18*['AMP_Standards']))
ENLITEN_kinetics = ENLITEN_kinetics.assign(concentration = 6*3*['no_reporter']+18*['0mol/0pM']+6*3*['0mol/0aM']+18*['10^-16mol/1pM']+6*3*['1.25e-20mol/125aM']+18*['10^-15mol/10pM']+6*3*['1.25e-18mol/12.5fM']+18*['10^-14mol/100pM']+6*3*['1.25e-16mol/1.25pM']+18*['10^-13mol/1nM']+6*3*['1.25e-15mol/12.5pM']+18*['10^-12mol/10nM']+6*3*['1.25e-14mol/125pM']+18*['10^-11mol/100nM']+6*3*['1.25e-13mol/1.25nM']+18*['10^-10mol/1uM'])
ENLITEN_kinetics = ENLITEN_kinetics.assign(concentration_moles = 6*3*['']+18*['0']+6*3*['0']+18*['1e-16']+6*3*['1.25e-20']+18*['1e-15']+6*3*['1.25e-18']+18*['1e-14']+6*3*['1.25e-16']+18*['1e-13']+6*3*['1.25e-15']+18*['1e-12']+6*3*['1.25e-14']+18*['1e-11']+6*3*['1.25e-13']+18*['1e-10'])

ENLITEN_kinetics.head(50)

,experiment,read,read_number,row,column,well,rfu_value,standard_substrate,kit,sample_set,concentration,concentration_moles
0,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,A,1,A1,687500,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
1,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,A,1,A1,683220,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
2,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,A,1,A1,673367,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
3,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,A,1,A1,673604,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
4,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,5,A,1,A1,665821,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
5,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,6,A,1,A1,660901,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
6,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,A,2,A2,666868,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
7,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,A,2,A2,660992,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
8,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,A,2,A2,650610,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,
9,NANCY_DREW,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,A,2,A2,655420,AMP,AMP_no_kinase_ENLITEN,NANCY_DREW,no_reporter,


In [43]:
ENLITEN_kinetics['rfu_value'] = pd.to_numeric(ENLITEN_kinetics['rfu_value'])
ENLITEN_kinetics.hvplot.bar(x = 'well', y = 'rfu_value', groupby = 'read_number', rot = 90,logy=True)

:DynamicMap   [read_number]
   :Bars   [well]   (rfu_value)

In [44]:
before = read_Promega_kinetics("../raw_txt_files/DETECTR_049/pre-clean_amyl-detectr049_1-9-2020_6-57-16 PM.csv", "Clean_Test")
before = before.assign(standard_substrate = len(before)*['ATP'])
before = before.assign(kit = len(before)*['ENLITEN'])
before = before.assign(timepoint = len(before)*['before_clean'])

before.tail(10)

,experiment,read,read_number,row,column,well,rfu_value,standard_substrate,kit,timepoint
8,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,A,11,A11,51555.4,ATP,ENLITEN,before_clean
9,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,A,11,A11,50092,ATP,ENLITEN,before_clean
10,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,5,A,11,A11,49292.1,ATP,ENLITEN,before_clean
11,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,6,A,11,A11,48179.3,ATP,ENLITEN,before_clean
12,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,A,12,A12,50430.5,ATP,ENLITEN,before_clean
13,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,A,12,A12,49941.9,ATP,ENLITEN,before_clean
14,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,A,12,A12,48163.6,ATP,ENLITEN,before_clean
15,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,A,12,A12,47646.9,ATP,ENLITEN,before_clean
16,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,5,A,12,A12,46554.9,ATP,ENLITEN,before_clean
17,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,6,A,12,A12,45557.1,ATP,ENLITEN,before_clean


In [45]:
after = read_Promega_kinetics("../raw_txt_files/DETECTR_049/aft-clean_amyl-detectr049_1-9-2020_8-55-22 PM.csv", "Clean_Test")
after = after.assign(standard_substrate = len(after)*['ATP'])
after = after.assign(kit = len(after)*['ENLITEN'])
after = after.assign(timepoint = len(after)*['after_clean'])

after.tail(10)

,experiment,read,read_number,row,column,well,rfu_value,standard_substrate,kit,timepoint
8,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,C,11,C11,13519.9,ATP,ENLITEN,after_clean
9,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,C,11,C11,13265.3,ATP,ENLITEN,after_clean
10,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,5,C,11,C11,13258.5,ATP,ENLITEN,after_clean
11,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,6,C,11,C11,12805.2,ATP,ENLITEN,after_clean
12,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,C,12,C12,12657.8,ATP,ENLITEN,after_clean
13,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,C,12,C12,12248.9,ATP,ENLITEN,after_clean
14,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,C,12,C12,12081.3,ATP,ENLITEN,after_clean
15,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,C,12,C12,11800.1,ATP,ENLITEN,after_clean
16,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,5,C,12,C12,11718.2,ATP,ENLITEN,after_clean
17,Clean_Test,100µl per well (Injector 1) 2sec. 1 repeat. 96...,6,C,12,C12,11436.5,ATP,ENLITEN,after_clean


In [54]:
ATP_stds = read_Promega_kinetics("../raw_txt_files/DETECTR_049/atp-stds_amyl-detectr049_1-9-2020_10-32-59 PM.csv", "ATP_Standards")
ATP_stds = ATP_stds.assign(standard_substrate = len(ATP_stds)*['ATP'])
ATP_stds = ATP_stds.assign(kit = len(ATP_stds)*['ENLITEN'])
ATP_stds = ATP_stds.assign(concentration = 18*['0mol/0pM']+18*['10^-17mol/100fM']+18*['10^-16mol/1pM']+18*['10^-15mol/10pM']+18*['10^-14mol/100pM']+18*['10^-13mol/1nM']+18*['10^-12mol/10nM']+18*['10^-11mol/100nM'])
    
ATP_stds = ATP_stds.assign(concentration_moles = 18*['0']+18*['1e-17']+18*['1e-16']+18*['1e-15']+18*['1e-14']+18*['1e-13']+18*['1e-12']+18*['1e-11'])

ATP_stds.head(50)

,experiment,read,read_number,row,column,well,rfu_value,standard_substrate,kit,concentration,concentration_moles
0,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,A,7,A7,3738.19,ATP,ENLITEN,0mol/0pM,0
1,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,A,7,A7,3733.99,ATP,ENLITEN,0mol/0pM,0
2,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,A,7,A7,3798.41,ATP,ENLITEN,0mol/0pM,0
3,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,A,7,A7,3681.87,ATP,ENLITEN,0mol/0pM,0
4,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,5,A,7,A7,3676.87,ATP,ENLITEN,0mol/0pM,0
5,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,6,A,7,A7,3643.76,ATP,ENLITEN,0mol/0pM,0
6,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,A,8,A8,3899.74,ATP,ENLITEN,0mol/0pM,0
7,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,A,8,A8,3925.85,ATP,ENLITEN,0mol/0pM,0
8,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,A,8,A8,3789.1,ATP,ENLITEN,0mol/0pM,0
9,ATP_Standards,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,A,8,A8,3703.28,ATP,ENLITEN,0mol/0pM,0


In [61]:
DETECTR_049 = pd.concat([ENLITEN_kinetics,before,after,ATP_stds])

/Users/amy.lyden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [65]:
DETECTR_049.columns

Index(['column', 'concentration', 'concentration_moles', 'experiment', 'kit',
       'read', 'read_number', 'rfu_value', 'row', 'sample_set',
       'standard_substrate', 'timepoint', 'well'],
      dtype='object')

In [66]:
len(DETECTR_049)

468

In [67]:
DETECTR_049.head(10)

,column,concentration,concentration_moles,experiment,kit,read,read_number,rfu_value,row,sample_set,standard_substrate,timepoint,well
0,1,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,687500,A,NANCY_DREW,AMP,NaN,A1
1,1,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,683220,A,NANCY_DREW,AMP,NaN,A1
2,1,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,673367,A,NANCY_DREW,AMP,NaN,A1
3,1,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,673604,A,NANCY_DREW,AMP,NaN,A1
4,1,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,5,665821,A,NANCY_DREW,AMP,NaN,A1
5,1,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,6,660901,A,NANCY_DREW,AMP,NaN,A1
6,2,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,1,666868,A,NANCY_DREW,AMP,NaN,A2
7,2,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,2,660992,A,NANCY_DREW,AMP,NaN,A2
8,2,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,3,650610,A,NANCY_DREW,AMP,NaN,A2
9,2,no_reporter,,NANCY_DREW,AMP_no_kinase_ENLITEN,100µl per well (Injector 1) 2sec. 1 repeat. 96...,4,655420,A,NANCY_DREW,AMP,NaN,A2


In [68]:
DETECTR_049.to_csv("../tidy_data/DETECTR_049_tidy.csv")